# Maximum common substructure (MCS)

MolecularGraph.jl version: 0.10.0

MolecularGraph.js implements essential MCS methods for cheminformatics applications.

- Node induced (MCIS) / Edge induced (MCES)
- Connected / disconnected
- Topological constraint (tdMCS)
- Graph-based local similarity (GLS)


In [1]:
using Pkg
Pkg.activate("..")
using MolecularGraph
using MolecularGraph.Graph
using Profile

  Activating environment at `~/Repository/MolecularGraph.jl_notebook/Project.toml`
Path `../../MolecularGraph.jl` exists and looks like the correct package. Using existing path.
   Resolving package versions...
  No Changes to `~/Repository/MolecularGraph.jl_notebook/Project.toml`
  No Changes to `~/Repository/MolecularGraph.jl_notebook/Manifest.toml`
┌ Info: Precompiling MolecularGraph [6c89ec66-9cd8-5372-9f91-fabc50dd27fd]
└ @ Base loading.jl:1317


In [2]:
# Download test data from PubChem

# PubChem ID->Name
compounds = Dict(
    "6047" => "Levodopa",
    "74217" => "3-Aminocoumarin",
    "6437877" => "Cefditoren Pivoxil",
    "5481173" => "Ceftazidime"
)

# Create data directory
data_dir = "_data"
isdir(data_dir) || mkdir(data_dir)

# Fetch
for (cid, name) in compounds
    url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/$(cid)/SDF"
    dest = joinpath(data_dir, "$(name).mol")
    isfile(dest) || download(url, dest);
end

# Convenient function to display a pair of mol images
function displayimgpair(img1, img2)
    """<div>
        <div style="float:left">$(img1)</div>
        <div style="float:left">$(img2)</div>
    </div>"""
end

displayimgpair (generic function with 1 method)

- Let's try MCS functionalities with very small molecules.
- Drop hydrogens from downloaded molecule data by `removehydrogens(mol, all=true)`

In [3]:
ldopa = sdftomol(joinpath(data_dir, "Levodopa.mol"))
ldopa = removehydrogens(ldopa, all=true)
precalculate!(ldopa)
svg1 = drawsvg(ldopa, 200, 200)

aminocoumarin = sdftomol(joinpath(data_dir, "3-Aminocoumarin.mol"))
aminocoumarin = removehydrogens(aminocoumarin, all=true)
precalculate!(aminocoumarin)
svg2 = drawsvg(aminocoumarin, 200, 200)

display("text/html", displayimgpair(svg1, svg2))

NH 2 
 HO 
 O 
 OH 
 OH 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 O 
 NH 2

## Maximum common induced substructure (MCIS)

- `disconnectedmcis` method calculates maximum common induced substructure (MCIS).
- `disconnectedmcis` returns a `MaxCommonSubgraphResult` type object.
- `mapping` (type `Dict{Int,Int}`) is a isomorphisim mapping of node(atom) indices between two molecules.
- `nodesubgraph` can be used to retrieve the matched subgraph and then display the substructure.
- `status`(type `Symbol`) shows the calculation status. `:done` means that the calculation was successfully finished.
- Use `drawsvg` with `highlight` keyword to display the matched substructure.

In [4]:
result = disconnectedmcis(ldopa, aminocoumarin)

subg1 = nodesubgraph(ldopa, Set(keys(result.mapping)))
svg1 = drawsvg(ldopa, 200, 200, highlight=subg1)

subg2 = nodesubgraph(aminocoumarin, Set(values(result.mapping)))
svg2 = drawsvg(aminocoumarin, 200, 200, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(result.status)")
println("MCIS size: $(length(result.mapping))")

NH 2 
 HO 
 O 
 OH 
 OH 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 O 
 NH 2

Status: done
MCIS size: 8


- Default node matcher function `nodematch` compares two atoms by their atomic symbol (`:C`, `:O`, `:N`, ...) and the number of $\pi$ electrons.
- The number of $\pi$ electrons gives as more intrinsic information of 3D geometry of the molecule than aparent bond order that cannot deal with conjugated system of the molecular orbital.
- Other atomic properties (charges, multiplicity, isotope, ...) of atoms are ignored by default.
- You can customize `nodematch` function and pass it to `disconnectedmcis` as an optional argument `nodematcher`.

## Maximum common edge-induced substructure (MCES)

- `disconnectedmces` calculates maximum common edge-induced substructure (MCES).
- Many chemists may feel that MCES is more intuitive than MCIS.
- `disconnectedmces` also returns a `MaxCommonSubgraphResult` type object.
- Note that the result is an mapping of edge indices between two molecules. So use `edgesubgraph` to show the matched subgraph.

In [5]:
result = disconnectedmces(ldopa, aminocoumarin)

subg1 = edgesubgraph(ldopa, Set(keys(result.mapping)))
svg1 = drawsvg(ldopa, 200, 200, highlight=subg1)

subg2 = edgesubgraph(aminocoumarin, Set(values(result.mapping)))
svg2 = drawsvg(aminocoumarin, 200, 200, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(result.status)")
println("MCES size: $(length(result.mapping))")

NH 2 
 HO 
 O 
 OH 
 OH 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 O 
 NH 2

Status: done
MCES size: 8


## Connected MCS

- `connectedmces` and `connectedmcis` calculate connected MCS.
- Connected MCS is far faster than disconnected MCS but tend to be smaller in size.
- As disconnected MCS does not care the spatial relationship among matched fragments, connected MCS can be more appropriate option for some kind of applications.

In [6]:
result = connectedmcis(ldopa, aminocoumarin)

subg1 = nodesubgraph(ldopa, Set(keys(result.mapping)))
svg1 = drawsvg(ldopa, 200, 200, highlight=subg1)

subg2 = nodesubgraph(aminocoumarin, Set(values(result.mapping)))
svg2 = drawsvg(aminocoumarin, 200, 200, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(result.status)")
println("Connected MCIS size: $(length(result.mapping))")

NH 2 
 HO 
 O 
 OH 
 OH 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 O 
 NH 2

Status: done
Connected MCIS size: 7


## Working with larger molecules

- Then, let's try it with a little larger molecules.

In [7]:
cefditoren = sdftomol(joinpath(data_dir, "Cefditoren Pivoxil.mol"))
cefditoren = removehydrogens(cefditoren, all=true)
svg1 = drawsvg(cefditoren, 250, 250)

ceftazidime = sdftomol(joinpath(data_dir, "Ceftazidime.mol"))
ceftazidime = removehydrogens(ceftazidime, all=true)
svg2 = drawsvg(ceftazidime, 250, 250)

display("text/html", displayimgpair(svg1, svg2))

O 
 NH 2 
 NH 
 O 
 S 
 O 
 N 
 O 
 N 
 S 
 O 
 O 
 N 
 N 
 S 
 O 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 N + 
 OH 
 S 
 O 
 NH 
 O 
 N 
 O 
 O 
 O – 
 N 
 NH 2 
 S 
 N

In [8]:
result = disconnectedmces(cefditoren, ceftazidime, timeout=10)

subg1 = edgesubgraph(cefditoren, Set(keys(result.mapping)))
svg1 = drawsvg(cefditoren, 250, 250, highlight=subg1)

subg2 = edgesubgraph(ceftazidime, Set(values(result.mapping)))
svg2 = drawsvg(ceftazidime, 250, 250, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(result.status)")
println("MCES size: $(length(result.mapping))")

O 
 NH 2 
 NH 
 O 
 S 
 O 
 N 
 O 
 N 
 S 
 O 
 O 
 N 
 N 
 S 
 O 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 N + 
 OH 
 S 
 O 
 NH 
 O 
 N 
 O 
 O 
 O – 
 N 
 NH 2 
 S 
 N

Status: timedout
MCES size: 25


- As disconnected MCS takes very long time,  keyword argument`timeout` was set to 10 second to abort MCS calculation.
- If it timed out, the status will be `timedout` and the result will be suboptimal common substructure size calculated so far.

In [9]:
@profile disconnectedmces(cefditoren, ceftazidime, timeout=10)
Profile.print(mincount=1000)

Overhead ╎ [+additional indent] Count File:Line; Function
    ╎6496 @Base/task.jl:406; (::IJulia.var"#15#18")()
    ╎ 6496 ...lia/src/eventloop.jl:8; eventloop(socket::ZMQ.Socket)
    ╎  6496 @Base/essentials.jl:706; invokelatest
    ╎   6496 @Base/essentials.jl:708; #invokelatest#2
    ╎    6496 ...execute_request.jl:67; execute_request(socket::ZMQ.S...
    ╎     6496 ...SoftGlobalScope.jl:65; softscope_include_string(m::...
    ╎    ╎ 6496 @Base/loading.jl:1094; include_string(mapexpr::typ...
   2╎    ╎  6496 @Base/boot.jl:360; eval
    ╎    ╎   6494 ...tructurematch.jl:426; (::MolecularGraph.var"#dis...
    ╎    ╎    6494 ...tructurematch.jl:426; #disconnectedmces#87
    ╎    ╎     6494 ...ructurematch.jl:419; maxcommonsubstruct##kw
    ╎    ╎    ╎ 6494 ...ructurematch.jl:421; maxcommonsubstruct(mol1:...
    ╎    ╎    ╎  6494 ...sm/cliquemcs.jl:49; (::MolecularGraph.Graph....
    ╎    ╎    ╎   6330 ...sm/cliquemcs.jl:75; maxcommonsubgraph(G::Gra...
    ╎    ╎    ╎    6330 ...raph/cl

- The most costful call was `maximalcliques` which yields maximal cliques. Maximum clique detection problem is known to be NP-hard.

- Connected MCS is far faster than disconnected MCS.

In [10]:
result = connectedmces(cefditoren, ceftazidime)

subg1 = edgesubgraph(cefditoren, Set(keys(result.mapping)))
svg1 = drawsvg(cefditoren, 250, 250, highlight=subg1)

subg2 = edgesubgraph(ceftazidime, Set(values(result.mapping)))
svg2 = drawsvg(ceftazidime, 250, 250, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(result.status)")
println("Connected MCES size: $(length(result.mapping))")

O 
 NH 2 
 NH 
 O 
 S 
 O 
 N 
 O 
 N 
 S 
 O 
 O 
 N 
 N 
 S 
 O 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 N + 
 OH 
 S 
 O 
 NH 
 O 
 N 
 O 
 O 
 O – 
 N 
 NH 2 
 S 
 N

Status: done
Connected MCES size: 26


- Or you can use `targetsize` keyword argument to know that these compounds have at least the given size of the common substructure. This feature is quite useful in library screening.

In [11]:
result = disconnectedmces(cefditoren, ceftazidime, targetsize=20)

subg1 = edgesubgraph(cefditoren, Set(keys(result.mapping)))
svg1 = drawsvg(cefditoren, 250, 250, highlight=subg1)

subg2 = edgesubgraph(ceftazidime, Set(values(result.mapping)))
svg2 = drawsvg(ceftazidime, 250, 250, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(result.status)")
println("Connected MCES size: $(length(result.mapping))")

O 
 NH 2 
 NH 
 O 
 S 
 O 
 N 
 O 
 N 
 S 
 O 
 O 
 N 
 N 
 S 
 O 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 N + 
 OH 
 S 
 O 
 NH 
 O 
 N 
 O 
 O 
 O – 
 N 
 NH 2 
 S 
 N

Status: targetreached
Connected MCES size: 20


## Topological constraint

- Disconnected MCS methods can detect as many matched fragments as possible, but it does not reflect spatial relationship of each fragments.
- On the other hand, connected MCS is too strict not to allow distant matches.
- Graph distance function can be used as a node attribute matcher to constrain the spatial arrangement of matched substructure fragments (topological constraint). This feature seems to be preferable in pharmacophore matching and structural similarity screening.
- Topological constraint also effectively decreases the calculation cost.
- Topological constraint can be used by passing keyword argument `topological=True`.
- Distance mismatch tolerance parameter $\theta$ is also available as the keyword argument `tolerance`.

In [12]:
result = tcmces(cefditoren, ceftazidime, tolerance=1)

subg1 = edgesubgraph(cefditoren, Set(keys(result.mapping)))
svg1 = drawsvg(cefditoren, 250, 250, highlight=subg1)

subg2 = edgesubgraph(ceftazidime, Set(values(result.mapping)))
svg2 = drawsvg(ceftazidime, 250, 250, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(result.status)")
println("tcMCES size: $(length(result.mapping))")

O 
 NH 2 
 NH 
 O 
 S 
 O 
 N 
 O 
 N 
 S 
 O 
 O 
 N 
 N 
 S 
 O 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 N + 
 OH 
 S 
 O 
 NH 
 O 
 N 
 O 
 O 
 O – 
 N 
 NH 2 
 S 
 N

Status: done
tcMCES size: 28
